In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from transformers import get_scheduler
from torch.optim import AdamW   
import torch
from tqdm.auto import tqdm
# from datasets import load_metric

c:\Users\bengo\funny_folder\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset("yelp_review_full", cache_dir='.')

Generating test split: 100%|██████████| 50000/50000 [00:00<00:00, 634840.76 examples/s]


In [3]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(1000))


In [4]:
tokenizer = AutoTokenizer.from_pretrained("Llama-encoder-1.0B")
tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = small_train_dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/1000 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 1000/1000 [00:00<00:00, 8094.22 examples/s]


In [5]:
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_train = tokenized_train.rename_column("label", "labels")


In [ ]:
train_dataloader = DataLoader(tokenized_train, shuffle=True, batch_size=1)
model = AutoModelForSequenceClassification.from_pretrained("Llama-encoder-1.0B", num_labels=5)
model.config.pad_token_id = model.config.eos_token_id
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

progress_bar = tqdm(range(num_training_steps))

model.train()
# model.model.gradient_checkpointing = True

for epoch in range(num_epochs):
    for batch in train_dataloader:

        preprocess_batch = {}
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                preprocess_batch[k] = v.to(device)
            elif isinstance(v, list):
                preprocess_batch[k] = torch.stack(v, dim=1).to(device)

        for k, v in preprocess_batch.items():
            print(k,v.shape)
        # batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**preprocess_batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at Llama-encoder-1.0B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/3000 [00:00<?, ?it/s]

labels torch.Size([1])
input_ids torch.Size([1, 110])
attention_mask torch.Size([1, 110])


  0%|          | 1/3000 [00:08<7:11:25,  8.63s/it]

labels torch.Size([1])
input_ids torch.Size([1, 90])
attention_mask torch.Size([1, 90])


  0%|          | 2/3000 [00:13<5:19:01,  6.38s/it]

...

labels torch.Size([1])
input_ids torch.Size([1, 124])
attention_mask torch.Size([1, 124])


  1%|          | 27/3000 [03:43<6:01:16,  7.29s/it]

labels torch.Size([1])
input_ids torch.Size([1, 377])
attention_mask torch.Size([1, 377])


  1%|          | 28/3000 [03:56<7:31:35,  9.12s/it]

labels torch.Size([1])
input_ids torch.Size([1, 529])
attention_mask torch.Size([1, 529])
